<a href="https://colab.research.google.com/github/kriskirla/AllProjects/blob/master/Algotrading/Portfolio_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Description: Analyze my portfolio

In [ ]:
pip install PyPortfolioOpt
pip install pulp

In [ ]:
# Import libraries 
import pandas_datareader as web
import pandas as pd
import numpy as np
import requests
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime as dt

In [ ]:
# Tickers to track (UPDATE THIS TO REFLECT)
ticker = ["ARKG", "ARKK", "AAPL", "AMD", "GME", "LSPD", "NIO", "ZOM"]

In [ ]:
# Get information about ticker
def get_company_information(symbol):
  ticker = yf.Ticker(symbol)
  return ticker.info

In [ ]:
# Get data from yahoo finance
df = web.DataReader(ticker, 'yahoo', start="2015-01-01", end=dt.today().strftime('%Y-%m-%d'))['Close']
df

Symbols,ARKG,ARKK,AAPL,AMD,GME,LSPD,NIO,ZOM
Date,,,,,,,,
2015-01-02,NaN,20.170000,27.332500,2.670000,33.799999,NaN,NaN,NaN
2015-01-05,NaN,19.684000,26.562500,2.660000,34.720001,NaN,NaN,NaN
2015-01-06,NaN,19.610001,26.565001,2.630000,33.689999,NaN,NaN,NaN
2015-01-07,NaN,19.700001,26.937500,2.580000,33.299999,NaN,NaN,NaN
2015-01-08,NaN,20.139999,27.972500,2.610000,33.689999,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2021-02-08,112.230003,153.309998,136.910004,91.470001,60.000000,72.010002,59.070000,2.70
2021-02-09,112.010002,154.929993,136.009995,90.910004,50.310001,72.800003,62.840000,2.32
2021-02-10,110.580002,153.100006,135.389999,92.349998,51.200001,71.300003,61.259998,2.55


In [ ]:
# Calculate MacD, simply for displaying
exp1 = df.ewm(span=12, adjust=False).mean()
exp2 = df.ewm(span=26, adjust=False).mean()
macd = exp1 - exp2
exp3 = macd.ewm(span=9, adjust=False).mean()
macd.plot(label='AAPL MACD', color='g')
ax = exp3.plot(label='Signal Line', color='r')
df.plot(ax=ax, secondary_y=True, label='AAPL')

The Parts above is to pull the holdings from S&P500

The parts below is to optimize portfolio

In [ ]:
# Optimize the portfolio
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [ ]:
# Calculate the expected annualized returns and the annualized sample covariance matrix of the daily asset returns
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)

# Optimize for the maximal Sharpe ratio
ef = EfficientFrontier(mu, S) # Create the Efficient Frontier Object
weights = ef.max_sharpe()
clean_weights = ef.clean_weights()
print(clean_weights)
ef.portfolio_performance(verbose=True)

OrderedDict([('ARKG', 0.0), ('ARKK', 0.0), ('AAPL', 0.0), ('AMD', 0.0), ('GME', 0.13275), ('LSPD', 0.85547), ('NIO', 0.0), ('ZOM', 0.01177)])
Expected annual return: 610.6%
Annual volatility: 51.2%
Sharpe Ratio: 11.89


/usr/local/lib/python3.6/dist-packages/pypfopt/risk_models.py:69: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  "The covariance matrix is non positive semidefinite. Amending eigenvalues."


(6.105898600717075, 0.5120267336058721, 11.885900093258881)

In [ ]:
# Get the discrete allocation of each share per stock
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

porfolio_val = 20000
latest_prices = get_latest_prices(df)
da = DiscreteAllocation(clean_weights, latest_prices, porfolio_val)
allocation, leftover = da.lp_portfolio()
print(f"Discrete Allocation: {allocation}\nFunds Remaining: {leftover}")

Discrete Allocation: {'GME': 51, 'LSPD': 234, 'ZOM': 92}
Funds Remaining: 4.480713844299316


In [ ]:
# Save information to print
company_name = []
industry = []
share_price = []
total_price = []

for symbol in allocation:
    info = get_company_information(symbol)
    company_name.append(info['longName'])
    industry.append(info['industry'])
    share_price.append(info['ask'])
    total_price.append(f"${info['ask'] * allocation[symbol]}")

In [ ]:
# Print the final guestimation of what to buy
portfolio_df = pd.DataFrame(columns=["Ticker", "Name", "Allocation", "Price/Share", "Total", "Industry"])
portfolio_df["Ticker"] = allocation.keys()
portfolio_df["Name"] = company_name
portfolio_df["Allocation"] = allocation.values()
portfolio_df["Price/Share"] = share_price
portfolio_df["Total"] = total_price
portfolio_df["Industry"] = industry
portfolio_df

,Ticker,Name,Allocation,Price/Share,Total,Industry
0,GME,GameStop Corp.,51,52.92,$2698.92,Specialty Retail
1,LSPD,Lightspeed POS Inc.,234,73.80,$17269.2,Software—Application
2,ZOM,Zomedica Corp.,92,2.57,$236.44,Drug Manufacturers—Specialty & Generic
